In [ ]:
import pandas as pd
import numpy as np

medsCohort = pd.read_stata("/Users/burke/Documents/research/bpCog/meds.dta")
medsCohort['antiHypertensiveCount'] = medsCohort[['medsbpace', 'medsbparb', 'medsbpalphabs', 'medsbpbetabs', 'medsbpcalbs', 'medsbpdiur', 'medsbprenin', 'medsbpvas', 'medsbpoth', 'afibslfrep', 'Hxafib', 'afibinc',]].sum(axis=1)
medsCohort['statin'] = medsCohort.medsstatinshchol
medsCohort['afib'] = medsCohort.index.isin(medsCohort.loc[(medsCohort.afibslfrep==1) |  (medsCohort.Hxafib == 1) | (medsCohort.afibinc == 1)].index)
medsCohort['afib'] = medsCohort['afib'].astype('int')

medsCohort= medsCohort[['newid', 'antiHypertensiveCount', 'statin', 'visitcounter', 'afib']]

completeCohort = pd.read_stata("/Users/burke/Documents/research/bpCog/combinedCohort.dta")

#completeCohort =  pd.read_csv("/Users/burke/Documents/research/bpCog/combinedCohort.csv", low_memory=False)

In [ ]:
completeCohort['newid'] = completeCohort['newid'].str[2:-1]

In [ ]:
len(completeCohort['newid'].unique())

In [ ]:
medsCohort.head()

In [5]:
completeCohort.drop(labels=['afibinc', 'hxafib', 'afibslfrep', 'ttoafib', 'afibdt', 'afib'], axis='columns', inplace=True)
completeCohort = completeCohort.merge(medsCohort, how='left', on=['newid', 'visitcounter'])

In [6]:
completeCohort.drop(labels=['sbpavg', 'dbpavg'], axis='columns', inplace=True)
completeCohort['sbpavg'] = completeCohort[['sbp1', 'sbp2', 'sbp3']].mean(axis=1)
completeCohort['dbpavg'] = completeCohort[['dbp1', 'dbp2', 'dbp3']].mean(axis=1)

In [7]:
completeCohort.racebpcog.value_counts()

2    177722
1     61078
3     36458
9      5528
Name: racebpcog, dtype: int64

In [8]:
cohort = completeCohort[['newid', 'visitcounter', 'sbpavg', 'dbpavg', 'bmi', 'trig', 'smokestatus', 'choltot', 'cholhdl', 'cholldl', 
                 'age0', 'female0', 'educ0', 'daysfromvisit1', 'racebpcog', 'hba1c', 'glucosef', 'educ0', 'physact', 'waistcm',
                 'antiHypertensiveCount', 'statin', 'afib', 'alcperwk']]

# build a clean new index in the cohort using the newid groups
cohort = cohort.assign(id=(cohort['newid']).astype('category').cat.codes)
cohort.drop(labels=['newid'], axis='columns', inplace=True)

cohort.rename(columns={'sbpavg' : 'sbp', 'dbpavg' : 'dbp', 'smokestatus' : 'smokingStatus', 'choltot' : 'totChol',
                      "cholhdl" : 'hdl', 'cholldl' : 'ldl', 'age0' : 'baseAge', 'female0' : 'gender', 'educ0' : 'education',
                      'hba1c' : 'a1c', 'racebpcog' : 'raceEthnicity', 'waistcm' : 'waist', 'physact' : 'anyPhysicalActivity', 'alcperwk' : 'alcoholPerWeek'}, inplace=True)


cohort = cohort.loc[:,~cohort.columns.duplicated()]


cohort.gender.replace(to_replace=[0,1], value=[1, 2], inplace=True)
# have to compress "hispanic" to "other hispanic" and no representation for mexican american in BP Cog
cohort.raceEthnicity = cohort.raceEthnicity.replace({1 : 4, 2 : 3,3 : 2,9 : 5}) 

In [9]:
cohort.alcoholPerWeek.describe()

count    179894.000000
mean         30.235582
std          94.532295
min           0.000000
25%           0.000000
50%           0.000000
75%          15.000000
max        5228.189941
Name: alcoholPerWeek, dtype: float64

In [10]:
allFactorsImputation = ["a1c", "hdl", "totChol", "bmi", "dbp", "sbp", "ldl", "trig", "glucosef", "waist", "anyPhysicalActivity", 'afib', 'antiHypertensiveCount', 'statin', 'alcoholPerWeek']

lagVars = []

# first geneate lag variables which we'll use in teh imputation
for factor in allFactorsImputation:
    newVarName = "lag" + factor[0].upper()+factor[1:]
    lagVars.append(newVarName)
    cohort[newVarName] = cohort.groupby(['id'])[factor].shift(1)
    cohort = cohort.join(other=cohort.groupby(['id'])[newVarName].mean(), on='id', rsuffix='mean')
    cohort.rename(columns={newVarName + "mean" : 'mean' + newVarName[0].upper() + newVarName[1:]}, inplace=True)
cohort.drop(lagVars, axis='columns', inplace=True)

cohort['age'] = cohort.baseAge + cohort.daysfromvisit1 / 365
cohort = pd.concat([cohort, pd.get_dummies(cohort['raceEthnicity'], prefix="raceEth")], axis=1)
cohort = pd.concat([cohort, pd.get_dummies(cohort['smokingStatus'], prefix="smoke")], axis=1)
cohort = pd.concat([cohort, pd.get_dummies(cohort['education'], prefix="educ")], axis=1)


cohort.drop(['baseAge', 'daysfromvisit1', 'visitcounter', 'smokingStatus', 'raceEthnicity', 'education'], axis='columns', inplace=True)

cohort.rename(columns={'raceEth_2' : 'raceEth2', 'raceEth_3' : 'raceEth3', 'raceEth_4' : 'raceEth4',
                        'raceEth_5' : 'raceEth5', 'smoke_0.0' : 'smoke0', 'smoke_1.0' : 'smoke1' , 
                       'smoke_2.0' : 'smoke2', 'educ_1.0' : 'educ1', 'educ_2.0' : 'educ2', 'educ_3.0' : 'educ3',
                      'educ_4.0' : 'educ4', 'educ_5.0' : 'educ5'}, inplace=True)


In [12]:
cohort.alcoholPerWeek.isna().sum()

100892

In [13]:
import statsmodels.imputation.mice as mice

def getFormulaForVariable(var):
    cols = list(cohort.columns)
    cols.remove(var)
    colStrings = [col + "+"for col in cols]
    return "".join(colStrings)[:-1]
    
imputedCohort = mice.MICEData(cohort)
#binaryVars = ['anyPhysicalActivity', 'smoke0', 'smoke1', 'smoke2', 'raceEth2', 'raceEth3', 'raceEth4', 'raceEth5', 'educ1', 'educ2', 'educ3', 'educ4', 'educ5']
#for var in binaryVars: 
#    imputedCohort.set_imputer(var, formula=getFormulaForVariable(var), model_class="logit")
imputedCohort.update_all(20)

In [ ]:
%matplotlib inline
imputedCohort.plot_fit_obs("a1c")

In [16]:
imputedCohort.data.alcoholPerWeek.describe()

count    280786.000000
mean         21.912180
std          80.406990
min           0.000000
25%           0.000000
50%           0.000000
75%           7.000000
max        5228.189941
Name: alcoholPerWeek, dtype: float64

In [17]:
imputedData = imputedCohort.data.copy()

# rebvuild categoricals
imputedData['smokingStatus'] = imputedData[['smoke0', 'smoke1', 'smoke2']].idxmax(axis=1)
imputedData['smokingStatus'] = imputedData['smokingStatus'].str[-1:]
imputedData['raceEthnicity'] = imputedData[['raceEth2', 'raceEth3', 'raceEth4', 'raceEth5']].idxmax(axis=1)
imputedData['raceEthnicity'] = imputedData['raceEthnicity'].str[-1:]

In [18]:
# drop the dummies
imputedData.drop(['smoke0', 'smoke1','smoke2'], axis='columns', inplace=True)
imputedData.drop(['raceEth2', 'raceEth3','raceEth3', 'raceEth4', 'raceEth5'], axis='columns', inplace=True)
imputedData.drop(['educ1', 'educ2','educ3', 'educ4', 'educ5'], axis='columns', inplace=True)
imputedData.drop(list(imputedData.filter(regex='mean')), axis='columns', inplace=True)
# drop the mean variables (they were partially imputed)

In [19]:
# rebuild the mean variables off the impuated values
allFactorsImputation = ["a1c", "hdl", "totChol", "bmi","ldl", "trig", "glucosef",  "waist", "anyPhysicalActivity", "antiHypertensiveCount", "statin", "afib", 'alcoholPerWeek']
logFactorsImputation = ["dbp", "sbp"]

In [20]:
for factor in allFactorsImputation:
    newVarName = "lag" + factor[0].upper()+factor[1:]
    imputedData[newVarName] = imputedData.groupby(['id'])[factor].shift(1)
    imputedData = imputedData.join(other=imputedData.groupby(['id'])[newVarName].mean(), on='id', rsuffix='mean')
    imputedData.rename(columns={newVarName + "mean" : 'mean' + newVarName[0].upper() + newVarName[1:]}, inplace=True)

In [21]:
for factor in logFactorsImputation:
    newVarName = "lag" + factor[0].upper()+factor[1:]
    imputedData[newVarName] = imputedData.groupby(['id'])[factor].shift(1)
    logLagName = "log" + newVarName[0].upper() + newVarName[1:]
    logName = "log" + factor[0].upper() + factor[1:]

    imputedData[logName] = np.log(imputedData[factor])  
    imputedData[logLagName] = np.log(imputedData[newVarName])  
    imputedData = imputedData.join(other=imputedData.groupby(['id'])[logLagName].mean(), on='id', rsuffix='mean')
    imputedData.rename(columns={logLagName + "mean" : 'mean' + logLagName[0].upper() + logLagName[1:]}, inplace=True)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log
  import sys
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  


In [22]:
imputedData.raceEthnicity= imputedData.raceEthnicity.astype('category')
imputedData.smokingStatus= imputedData.smokingStatus.astype('category')
imputedData.sort_values(by=['id', 'age'], inplace=True)

In [23]:
imputedData.raceEthnicity.value_counts()

3    177722
4     61078
2     36458
5      5528
Name: raceEthnicity, dtype: int64

In [25]:
import statsmodels.formula.api as statsmodel
import sys
import os
sys.path.append(os.path.abspath("../mcm/"))
from mcm.regression_model import RegressionModel

regResults = {}

def buildAndExportRegressionModelForDataset(outcomeVariable , fullDataset, hasLog): 
    factors = allFactors.copy()
    factors.extend(logFactors)
    factors.remove(outcomeVariable)
    factors = ["mean" + factor[0].upper() + factor[1:] + " + " + factor + " + " for factor in factors] 
    
    meanLagOutcomeVariable =  "mean" + outcomeVariable[0].upper() + outcomeVariable[1:]
    
    if (not hasLog):
        outcomeVariable = outcomeVariable[3:4].lower() + outcomeVariable[4:] # remove the lag
    else:
        outcomeVariable = "log" + outcomeVariable[-3:] # remove the lag and add the log
    
    formula = outcomeVariable + " ~ " + "".join(factors) + "age + gender + smokingStatus + raceEthnicity + " + meanLagOutcomeVariable

    print("outcome: " + outcomeVariable + " hasLog: " + str(hasLog) + " : " +formula)

    model = statsmodel.ols(formula=formula, data=fullDataset)
    results = model.fit()
    regResults[outcomeVariable] = results
    mcmRegressionModel = RegressionModel(results.params.to_dict(), results.bse.to_dict(), results.resid.mean(), results.resid.std())
    mcmRegressionModel.write_json(os.path.abspath("../mcm/mcm/data/" + outcomeVariable + "CohortModelSpec.json"))
    #results.save("/Users/burke/Documents/research/bpCog/mcm/mcm/data/" + outcomeVariable +  "CohortModel.pickle")
    return results


allFactors = ["lagA1c", "lagHdl", "lagTotChol", "lagBmi", "lagLdl", "lagTrig", "lagWaist", "lagAnyPhysicalActivity", "lagAntiHypertensiveCount", "lagStatin", "lagAfib", 'lagAlcoholPerWeek']
logFactors = ["logLagDbp", "logLagSbp"]

imputedData['logDbp'].loc[imputedData['logDbp'] <= 0] = imputedData['logDbp'].loc[imputedData['logDbp']> 0].min()
imputedData['logLagDbp'].loc[imputedData['logLagDbp'] <= 0] = imputedData['logLagDbp'].loc[imputedData['logLagDbp']> 0].min()
imputedData['meanLogLagDbp'].loc[imputedData['meanLogLagDbp'] <= 0] = imputedData['meanLogLagDbp'].loc[imputedData['meanLogLagDbp']> 0].min()



for factor in allFactors:
    buildAndExportRegressionModelForDataset(factor, imputedData, False)
for factor in logFactors:
    buildAndExportRegressionModelForDataset(factor, imputedData, True)
    

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


outcome: a1c hasLog: False : a1c ~ meanLagHdl + lagHdl + meanLagTotChol + lagTotChol + meanLagBmi + lagBmi + meanLagLdl + lagLdl + meanLagTrig + lagTrig + meanLagWaist + lagWaist + meanLagAnyPhysicalActivity + lagAnyPhysicalActivity + meanLagAntiHypertensiveCount + lagAntiHypertensiveCount + meanLagStatin + lagStatin + meanLagAfib + lagAfib + meanLagAlcoholPerWeek + lagAlcoholPerWeek + meanLogLagDbp + logLagDbp + meanLogLagSbp + logLagSbp + age + gender + smokingStatus + raceEthnicity + meanLagA1c
outcome: hdl hasLog: False : hdl ~ meanLagA1c + lagA1c + meanLagTotChol + lagTotChol + meanLagBmi + lagBmi + meanLagLdl + lagLdl + meanLagTrig + lagTrig + meanLagWaist + lagWaist + meanLagAnyPhysicalActivity + lagAnyPhysicalActivity + meanLagAntiHypertensiveCount + lagAntiHypertensiveCount + meanLagStatin + lagStatin + meanLagAfib + lagAfib + meanLagAlcoholPerWeek + lagAlcoholPerWeek + meanLogLagDbp + logLagDbp + meanLogLagSbp + logLagSbp + age + gender + smokingStatus + raceEthnicity + meanL

In [26]:
imputedData.anyPhysicalActivity.value_counts(normalize=True)

1.0    0.812469
0.0    0.187531
Name: anyPhysicalActivity, dtype: float64

In [27]:
regResults['logSbp'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 logSbp   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.629
Method:                 Least Squares   F-statistic:                 1.194e+04
Date:                Tue, 26 Nov 2019   Prob (F-statistic):               0.00
Time:                        12:34:43   Log-Likelihood:             2.2870e+05
No. Observations:              239623   AIC:                        -4.573e+05
Df Residuals:                  239588   BIC:                        -4.570e+05
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        0.0831      0.009      9.731      0.000       0.066       0.100
smokingStatus[T.1]               0.0026      0.000      5.321      0.000       0.002       0.004
smokingStatus[T.2]               0.0029      0.001      4.363      0.000       0.002       0.004
raceEthnicity[T.3]              -0.0030      0.001     -4.670      0.000      -0.004      -0.002
raceEthnicity[T.4]              -0.0015      0.001     -2.046      0.041      -0.003   -6.26e-05
raceEthnicity[T.5]               0.0033      0.001      2.205      0.027       0.000       0.006
meanLagA1c                       0.0011      0.000      3.375      0.001       0.000       0.002
lagA1c                          -0.0005      0.000     -1.839      0.066      -0.001    3.35e-05
meanLagHdl                       0.0005      0.000      2.217      0.027    5.71e-05       0.001
lagHdl                        1.599e-05      0.000      0.126      0.900      -0.000       0.000
meanLagTotChol                  -0.0005      0.000     -2.475      0.013      -0.001      -0.000
lagTotChol                       0.0001      0.000      1.079      0.280      -0.000       0.000
meanLagBmi                      -0.0007      0.000     -3.975      0.000      -0.001      -0.000
lagBmi                          -0.0006      0.000     -4.506      0.000      -0.001      -0.000
meanLagLdl                       0.0006      0.000      2.632      0.008       0.000       0.001
lagLdl                       -5.395e-05      0.000     -0.446      0.656      -0.000       0.000
meanLagTrig                      0.0001   4.22e-05      2.444      0.015    2.04e-05       0.000
lagTrig                      -2.418e-05    2.3e-05     -1.052      0.293   -6.92e-05    2.09e-05
meanLagWaist                     0.0007   5.45e-05     12.353      0.000       0.001       0.001
lagWaist                        -0.0002   4.11e-05     -3.952      0.000      -0.000   -8.19e-05
meanLagAnyPhysicalActivity      -0.0032      0.001     -2.651      0.008      -0.006      -0.001
lagAnyPhysicalActivity           0.0002      0.001      0.221      0.825      -0.002       0.002
meanLagAntiHypertensiveCount     0.0057      0.000     11.343      0.000       0.005       0.007
lagAntiHypertensiveCount        -0.0066      0.000    -17.048      0.000      -0.007      -0.006
meanLagStatin                   -0.0004      0.001     -0.306      0.759      -0.003       0.002
lagStatin                        0.0021      0.001      2.282      0.022       0.000       0.004
meanLagAfib                      0.0083      0.002      3.690      0.000       0.004       0.013
lagAfib                         -0.0092      0.002     -5.140      0.000      -0.013      -0.006
meanLagAlcoholPerWeek        -4.042e-05   5.27e-06     -7.667      0.000   -

In [28]:
regResults['afib'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   afib   R-squared:                       0.656
Model:                            OLS   Adj. R-squared:                  0.656
Method:                 Least Squares   F-statistic:                 1.342e+04
Date:                Tue, 26 Nov 2019   Prob (F-statistic):               0.00
Time:                        12:34:57   Log-Likelihood:             1.5185e+05
No. Observations:              239623   AIC:                        -3.036e+05
Df Residuals:                  239588   BIC:                        -3.033e+05
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        0.0088      0.012      0.752      0.452      -0.014       0.032
smokingStatus[T.1]              -0.0040      0.001     -5.974      0.000      -0.005      -0.003
smokingStatus[T.2]               0.0004      0.001      0.446      0.656      -0.001       0.002
raceEthnicity[T.3]               0.0081      0.001      9.338      0.000       0.006       0.010
raceEthnicity[T.4]               0.0047      0.001      4.659      0.000       0.003       0.007
raceEthnicity[T.5]               0.0030      0.002      1.453      0.146      -0.001       0.007
meanLagA1c                      -0.0009      0.000     -1.891      0.059      -0.002    3.25e-05
lagA1c                           0.0010      0.000      2.735      0.006       0.000       0.002
meanLagHdl                      -0.0004      0.000     -1.316      0.188      -0.001       0.000
lagHdl                           0.0004      0.000      2.227      0.026    4.67e-05       0.001
meanLagTotChol                   0.0004      0.000      1.215      0.224      -0.000       0.001
lagTotChol                      -0.0005      0.000     -3.062      0.002      -0.001      -0.000
meanLagBmi                       0.0036      0.000     15.683      0.000       0.003       0.004
lagBmi                          -0.0019      0.000     -9.989      0.000      -0.002      -0.002
meanLagLdl                      -0.0003      0.000     -0.923      0.356      -0.001       0.000
lagLdl                           0.0004      0.000      2.252      0.024    4.88e-05       0.001
meanLagTrig                  -4.239e-05   5.82e-05     -0.728      0.467      -0.000    7.17e-05
lagTrig                       6.604e-05   3.17e-05      2.085      0.037    3.97e-06       0.000
meanLagWaist                    -0.0008   7.51e-05    -10.418      0.000      -0.001      -0.001
lagWaist                      6.585e-05   5.67e-05      1.162      0.245   -4.52e-05       0.000
meanLagAnyPhysicalActivity       0.0171      0.002     10.327      0.000       0.014       0.020
lagAnyPhysicalActivity          -0.0129      0.001     -8.994      0.000      -0.016      -0.010
meanLagAntiHypertensiveCount    -0.0635      0.001    -95.722      0.000      -0.065      -0.062
lagAntiHypertensiveCount         0.0635      0.001    125.226      0.000       0.063       0.065
meanLagStatin                   -0.0175      0.002    -10.451      0.000      -0.021      -0.014
lagStatin                        0.0151      0.001     11.948      0.000       0.013       0.018
meanLagAlcoholPerWeek         2.357e-05   7.27e-06      3.245      0.001    9.33e-06    3.78e-05
lagAlcoholPerWeek            -1.344e-06   5.75e-06     -0.234      0.815   -1.26e-05    9.93e-06
meanLogLagDbp                    0.0047      0.003      1.784      0.074    

In [29]:
regResults['alcoholPerWeek'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         alcoholPerWeek   R-squared:                       0.602
Model:                            OLS   Adj. R-squared:                  0.602
Method:                 Least Squares   F-statistic:                 1.064e+04
Date:                Tue, 26 Nov 2019   Prob (F-statistic):               0.00
Time:                        12:35:13   Log-Likelihood:            -1.2776e+06
No. Observations:              239623   AIC:                         2.555e+06
Df Residuals:                  239588   BIC:                         2.556e+06
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        6.8455      4.586      1.493      0.136      -2.144      15.835
smokingStatus[T.1]              -0.1017      0.262     -0.389      0.698      -0.614       0.411
smokingStatus[T.2]               3.1735      0.358      8.855      0.000       2.471       3.876
raceEthnicity[T.3]              -0.6674      0.340     -1.965      0.049      -1.333      -0.002
raceEthnicity[T.4]              -0.7782      0.390     -1.998      0.046      -1.542      -0.015
raceEthnicity[T.5]              -0.5921      0.803     -0.737      0.461      -2.166       0.982
meanLagA1c                       0.0646      0.182      0.355      0.723      -0.292       0.422
lagA1c                          -0.0654      0.148     -0.441      0.659      -0.356       0.225
meanLagHdl                      -0.0120      0.119     -0.100      0.920      -0.246       0.222
lagHdl                          -0.0152      0.068     -0.223      0.823      -0.149       0.118
meanLagTotChol                  -0.0417      0.117     -0.356      0.722      -0.272       0.188
lagTotChol                       0.0846      0.065      1.300      0.193      -0.043       0.212
meanLagBmi                       0.0774      0.089      0.865      0.387      -0.098       0.253
lagBmi                          -0.0434      0.073     -0.591      0.555      -0.187       0.101
meanLagLdl                       0.0202      0.118      0.172      0.864      -0.210       0.251
lagLdl                          -0.0682      0.065     -1.049      0.294      -0.196       0.059
meanLagTrig                     -0.0022      0.023     -0.098      0.922      -0.047       0.042
lagTrig                         -0.0088      0.012     -0.712      0.476      -0.033       0.015
meanLagWaist                    -0.0033      0.029     -0.113      0.910      -0.061       0.054
lagWaist                        -0.0277      0.022     -1.254      0.210      -0.071       0.016
meanLagAnyPhysicalActivity      -0.2755      0.647     -0.426      0.670      -1.544       0.993
lagAnyPhysicalActivity           1.2041      0.559      2.154      0.031       0.109       2.300
meanLagAntiHypertensiveCount     0.0953      0.268      0.356      0.722      -0.430       0.620
lagAntiHypertensiveCount         0.0840      0.210      0.401      0.689      -0.327       0.495
meanLagStatin                   -0.3506      0.654     -0.536      0.592      -1.633       0.932
lagStatin                        0.9894      0.491      2.015      0.044       0.027       1.952
meanLagAfib                      0.0762      1.204      0.063      0.950      -2.283       2.435
lagAfib                          0.3487      0.962      0.362      0.717      -1.537       2.235
meanLogLagDbp                   -2.6744      1.038     -2.577      0.010    

ok...mdels are looking a bit bette rnow. sbp model looks reasonbly close to stata model which is reassureing.

other thought...
if we systematically name the variables here, it may be relatively easy to build a risk model apparatus that reads from these files and implements them. so, "log" means that the predictor is log-transformed. "meanLab" means that you take the maen of all prior values. "lag" means that you take the the value immediately before.

then you can include variable names in snake case — so each variable is multiple in. you'd be able to specify multi-way interactions or quadratic terms pretty easily that way... 